Importing the necessary library

In [1]:

import gradio as gr
import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

c:\Users\PRINCE VERMA\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initialize the sentence tranformer

In [2]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

with open("embeddings.pkl", "rb") as f:
  embeddings = pickle.load(f)
embeddings = embeddings.astype("float32")

embedding_size = embeddings.shape[1]
print(embedding_size)

c:\Users\PRINCE VERMA\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


768


Create Faiss and insert embeddings of the images

In [3]:
n_clusters = 1
num_results = 5

quantizer = faiss.IndexFlatIP(embedding_size)
index = faiss.IndexIVFFlat(
    quantizer, embedding_size, n_clusters, faiss.METRIC_INNER_PRODUCT,
)

index.train(embeddings)
index.add(embeddings)

Search for the required image

In [5]:
def _search(query):
  query_embedding = model.encode(query)
  query_embedding = np.array(query_embedding).astype("float32")
  query_embedding = query_embedding.reshape(1, -1)
  _, indices = index.search(query_embedding, num_results)
  images = [f"./images/image-{i+1}.jpg" for i in indices[0]]
  return images

Using gradio for the better visualization

In [6]:
with gr.Blocks() as demo:
  query = gr.Textbox(lines = 1, label = "search query")
  outputs = gr.Gallery(preview = True)
  submit = gr.Button(value = "search")
  submit.click(_search, inputs = query, outputs = outputs)

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
